In [ ]:
import numpy as np
import tifffile as tiff
from skimage import filters
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt

# Define wavelength range
num_bands = 204         # মোট কতগুলো spectral band আছে (ডেটাসেট অনুযায়ী সেট করতে হবে)
wavelength_start = 400  # wavelength এর শুরু (nanometer এ)
wavelength_end = 1000   # wavelength এর শেষ (nanometer এ)

wavelengths = np.linspace(wavelength_start, wavelength_end, num_bands)  # 400 থেকে 1000 এর মধ্যে সমান ভাগে 204 টা wavelength বানাবে

# Function to calculate band metrics
def calculate_band_metrics(band_array):
    band_norm = (band_array - np.nanmin(band_array)) / (np.nanmax(band_array) - np.nanmin(band_array) + 1e-8)  # normalize to [0,1]
    
    brightness = np.nanmean(band_norm)            # average intensity
    contrast = np.nanstd(band_norm)               # standard deviation = contrast
    sharpness = filters.laplace(band_norm).var()  # Laplacian filter দিয়ে edge strength measure
    
    highpass = band_norm - filters.gaussian(band_norm, sigma=1)  # Gaussian blur বাদ দিয়ে high-frequency অংশ
    noise = np.nanstd(highpass)                                # noise estimate
    
    signal = np.sqrt(np.nanmean(band_norm**2))                 # signal power
    snr = 20 * np.log10(signal / (noise + 1e-8))               # Signal-to-Noise Ratio (dB)
    
    return {
        "Brightness": brightness,
        "Contrast": contrast,
        "Sharpness": sharpness,
        "SNR(dB)": snr
    }

# Interactive function
def show_band_metrics_interactive(img_id, band_idx):
    file_path = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{img_id:05d}.tiff"  # image path বানানো হচ্ছে
    try:
        cube = tiff.imread(file_path)      # Hyperspectral cube load হবে (bands, rows, cols)
        bands, rows, cols = cube.shape     # cube shape বের করা
        
        print(f"Loaded Image ID: {img_id}")
        print(f"Image width (columns): {cols}")      # image width
        print(f"Image height (rows): {rows}")        # image height
        print(f"Number of bands (channels): {bands}")# কতগুলো spectral band
        print(f"Data type: {cube.dtype}")            # data type (uint16, float32 etc.)
        bit_depth = cube.dtype.itemsize * 8          # কত bit depth
        print(f"Bit depth: {bit_depth}\n")
        
        cube = np.transpose(cube, (1, 2, 0))         # (bands, rows, cols) → (rows, cols, bands)
        
        band_array = cube[:, :, band_idx]            # নির্দিষ্ট band array বের করা
        metrics = calculate_band_metrics(band_array) # metrics calculate
        
        if len(wavelengths) == bands:                # যদি wavelength list আর band সংখ্যা মিলে যায়
            band_wavelength = wavelengths[band_idx]  # ওই band এর wavelength বের করা
            print(f"Selected Band: {band_idx} | Wavelength: {band_wavelength:.2f} nm")
        else:
            print(f"Selected Band: {band_idx} | Wavelength info not available")
        
        print(f"Brightness: {metrics['Brightness']:.4f}")  # brightness print
        print(f"Contrast: {metrics['Contrast']:.4f}")      # contrast print
        print(f"Sharpness: {metrics['Sharpness']:.4f}")    # sharpness print
        print(f"SNR(dB): {metrics['SNR(dB)']:.4f}")        # SNR print
        
        plt.figure(figsize=(5,5))                          # ইমেজ show করার জন্য figure বানানো
        plt.imshow(band_array, cmap='gray')                # grayscale image show
        plt.title(f"Image ID: {img_id} | Band: {band_idx}")# title
        plt.axis("off")                                    # axis off
        plt.show()
        
    except FileNotFoundError:
        print(f"File for Image ID {img_id} not found.")    # যদি image না পাওয়া যায়

# Dataset info
min_id = 0
max_id = 2409  # dataset এ মোট কতগুলো image আছে (adjust করতে হবে)

first_img_file = f"D:/Z Education/University/4-1 Course/Thesis/Hyper leaf dataset/image/{min_id:05d}.tiff"  # প্রথম image এর path
first_cube = tiff.imread(first_img_file)               # প্রথম image load
total_bands = first_cube.shape[0]                      # মোট কতগুলো band আছে বের করা

# Interactive sliders
interact(
    show_band_metrics_interactive,
    img_id=IntSlider(min=min_id, max=max_id, step=1, value=min_id, description='Image ID'),      # Image ID slider
    band_idx=IntSlider(min=0, max=total_bands-1, step=1, value=0, description='Band')            # Band slider
)